使用minimize求难度及系数，确定不同题目数量时，最优的w

In [90]:
from __future__ import division
import os
import sys
import glob
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
%matplotlib inline
%precision 4
plt.style.use('ggplot')
np.random.seed(1234)
import pymc
import scipy.stats as stats
from scipy.optimize import minimize, rosen, rosen_der

In [91]:
n_items=100
n_stu=100
real_b1 = np.random.normal(0,1,(1,n_items))
real_theta1 = np.random.normal(0,1,(n_stu,1))
real_b2 = np.random.normal(0,1,(1,n_items))
real_theta2 = np.random.normal(0,1,(n_stu,1))
uv=np.random.random((n_stu,n_items))<1.0/ (1.0 + np.exp(-(real_theta1-real_b1)))/ (1.0 + np.exp(-(real_theta2-real_b2)))
mask=np.random.random(uv.shape)<0.7
def absm(a,b):
    return np.abs(a-b).mean()
b0_real = np.array([ee for ee in real_b1.flat])
b1_real = np.array([ee for ee in real_b2.flat])
t0_real = np.array([ee for ee in real_theta1.flat])
t1_real = np.array([ee for ee in real_theta2.flat])
a = np.array([1]*2*(n_stu+n_items))
start1 = n_stu
start2 = n_stu*2
start3 = n_stu*2+n_items
bnds = ((-3, 3), (-3, 3))*(n_stu+n_items)
y = uv
for w in [ii/1000. for ii in range(20)]:
    fun = lambda x: (np.log(1+ np.exp(-(np.array(x[:start1]).reshape(real_theta1.shape)-np.array([[x[start2:start3]]]))))+
                     np.log(1+ np.exp(-(np.array(x[start1:start2]).reshape(real_theta2.shape)-np.array([[x[start3:]]]))))-
                     (1-y)*np.log((1+ np.exp(-(np.array(x[:start1]).reshape(real_theta1.shape)-np.array([[x[start2:start3]]]))))*(1+ np.exp(-(np.array(x[start1:start2]).reshape(real_theta1.shape)-np.array([[x[start3:]]]))))-1)).sum()/n_items+w*((np.array([x])**2).sum())
#     fun = lambda x: (np.log(1+ np.exp(-(np.array([[ii] for ii in x[:start1]])-np.array([[x[start2:start3]]]))))+
#                      np.log(1+ np.exp(-(np.array([[ii] for ii in x[start1:start2]])-np.array([[x[start3:]]]))))-
#                      (1-y)*np.log((1+ np.exp(-(np.array([[ii] for ii in x[:start1]])-np.array([[x[start2:start3]]]))))*(1+ np.exp(-(np.array([[ii] for ii in x[start1:start2]])-np.array([[x[start3:]]]))))-1)).sum()/n_items+w*((np.array([[ii] for ii in x])**2).sum())
    res = minimize(fun, (a), bounds=bnds,method='L-BFGS-B')
    # print res.x
    t0_min = res.x[:start1]
    t1_min = res.x[start1:start2]
    b0_min = res.x[start2:start3]
    b1_min = res.x[start3:]
    print w,(absm(t0_min,t0_real)+absm(t1_min,t1_real)+absm(b0_min,b0_real)+absm(b1_min,b1_real))/4,absm(t0_min,t0_real),absm(t1_min,t1_real),absm(b0_min,b0_real),absm(b1_min,b1_real)
# plt.hist(t0_min,10, histtype='step', normed=False, label='post')

0.0 1.02128206343 1.01850986646 0.958544769565 1.10835651186 0.99971710585
0.001 0.605849786632 0.577821812481 0.651046602706 0.634206111526 0.560324619816
0.002 0.605526057684 0.580447111835 0.648732943858 0.633735773347 0.559188401697
0.003 0.605233616165 0.58282405549 0.646643473167 0.633240700147 0.558226235855
0.004 0.605046633177 0.585007433024 0.644796779809 0.632989484445 0.55739283543
0.005 0.605014814951 0.587247404882 0.643309983776 0.632867259232 0.556634611913
0.006 0.60511805654 0.589686894025 0.642092789764 0.632731684035 0.555960858334
0.007 0.605298743508 0.592217132788 0.640995585642 0.632570737548 0.555411518053
0.008 0.605819039393 0.594811003692 0.639979779797 0.633123803968 0.555361570113
0.009 0.606404104916 0.59744344388 0.639087701422 0.633689830855 0.555395443507
0.01 0.606998956537 0.599976070829 0.638358587382 0.634223989101 0.555437178837
0.011 0.607591141052 0.602377281346 0.637660967327 0.634838372498 0.555487943035
0.012 0.608300795255 0.604658502611 0.6

100,100的数据，w取0.01最好，误差均在0.6左右，可定w取0.01-0.04

In [92]:
n_items=200
n_stu=100
real_b1 = np.random.normal(0,1,(1,n_items))
real_theta1 = np.random.normal(0,1,(n_stu,1))
real_b2 = np.random.normal(0,1,(1,n_items))
real_theta2 = np.random.normal(0,1,(n_stu,1))
uv=np.random.random((n_stu,n_items))<1.0/ (1.0 + np.exp(-(real_theta1-real_b1)))/ (1.0 + np.exp(-(real_theta2-real_b2)))
def absm(a,b):
    return np.abs(a-b).mean()
b0_real = np.array([ee for ee in real_b1.flat])
b1_real = np.array([ee for ee in real_b2.flat])
t0_real = np.array([ee for ee in real_theta1.flat])
t1_real = np.array([ee for ee in real_theta2.flat])
a = np.array([1]*2*(n_stu+n_items))
start1 = n_stu
start2 = n_stu*2
start3 = n_stu*2+n_items
bnds = ((-3, 3), (-3, 3))*(n_stu+n_items)
y = uv
for w in [ii/1000. for ii in range(20)]:
    fun = lambda x: (np.log(1+ np.exp(-(np.array(x[:start1]).reshape(real_theta1.shape)-np.array([[x[start2:start3]]]))))+
                     np.log(1+ np.exp(-(np.array(x[start1:start2]).reshape(real_theta2.shape)-np.array([[x[start3:]]]))))-
                     (1-y)*np.log((1+ np.exp(-(np.array(x[:start1]).reshape(real_theta1.shape)-np.array([[x[start2:start3]]]))))*(1+ np.exp(-(np.array(x[start1:start2]).reshape(real_theta1.shape)-np.array([[x[start3:]]]))))-1)).sum()/n_items+w*((np.array([x])**2).sum())
#     fun = lambda x: (np.log(1+ np.exp(-(np.array([[ii] for ii in x[:start1]])-np.array([[x[start2:start3]]]))))+
#                      np.log(1+ np.exp(-(np.array([[ii] for ii in x[start1:start2]])-np.array([[x[start3:]]]))))-
#                      (1-y)*np.log((1+ np.exp(-(np.array([[ii] for ii in x[:start1]])-np.array([[x[start2:start3]]]))))*(1+ np.exp(-(np.array([[ii] for ii in x[start1:start2]])-np.array([[x[start3:]]]))))-1)).sum()/n_items+w*((np.array([[ii] for ii in x])**2).sum())
#     res = minimize(fun, (t0,t1,b0,b1), bounds=bnds,method='L-BFGS-B')
    res = minimize(fun, (a), bounds=bnds,method='L-BFGS-B')
    # print res.x
    t0_min = res.x[:start1]
    t1_min = res.x[start1:start2]
    b0_min = res.x[start2:start3]
    b1_min = res.x[start3:]
    print w,(absm(t0_min,t0_real)+absm(t1_min,t1_real)+absm(b0_min,b0_real)+absm(b1_min,b1_real))/4,absm(t0_min,t0_real),absm(t1_min,t1_real),absm(b0_min,b0_real),absm(b1_min,b1_real)

0.0 0.848886762238 0.847094028445 0.814094117114 0.894316323907 0.840042579488
0.001 0.583163213882 0.607707869832 0.597904421468 0.589484832098 0.537555732128
0.002 0.584464977714 0.611200083885 0.597515385752 0.590199436326 0.538945004893
0.003 0.586185305702 0.614784763022 0.597728554488 0.591979498031 0.540248407268
0.004 0.587877682158 0.61805537491 0.598227351582 0.593702906759 0.541525095383
0.005 0.589805437049 0.6210579003 0.598767823254 0.596270298091 0.543125726549
0.006 0.591841363662 0.623835636448 0.599379262415 0.599338616188 0.544811939597
0.007 0.593942077027 0.626432235143 0.59994262123 0.602559689113 0.546833762623
0.008 0.595976537556 0.628910184566 0.600461469571 0.605642106274 0.548892389814
0.009 0.598037903727 0.631243194219 0.601162152566 0.608525657898 0.551220610227
0.01 0.600060142388 0.633442272619 0.601947621454 0.6112697732 0.553580902278
0.011 0.601979544091 0.635529680359 0.602688924332 0.613830319717 0.555869251954
0.012 0.60380615203 0.637520776876 0.

In [93]:
n_items=300
n_stu=100
real_b1 = np.random.normal(0,1,(1,n_items))
real_theta1 = np.random.normal(0,1,(n_stu,1))
real_b2 = np.random.normal(0,1,(1,n_items))
real_theta2 = np.random.normal(0,1,(n_stu,1))
uv=np.random.random((n_stu,n_items))<1.0/ (1.0 + np.exp(-(real_theta1-real_b1)))/ (1.0 + np.exp(-(real_theta2-real_b2)))
def absm(a,b):
    return np.abs(a-b).mean()
b0_real = np.array([ee for ee in real_b1.flat])
b1_real = np.array([ee for ee in real_b2.flat])
t0_real = np.array([ee for ee in real_theta1.flat])
t1_real = np.array([ee for ee in real_theta2.flat])
a = np.array([1]*2*(n_stu+n_items))
start1 = n_stu
start2 = n_stu*2
start3 = n_stu*2+n_items
bnds = ((-3, 3), (-3, 3))*(n_stu+n_items)
y = uv
for w in [ii/1000. for ii in range(20)]:
    fun = lambda x: (np.log(1+ np.exp(-(np.array(x[:start1]).reshape(real_theta1.shape)-np.array([[x[start2:start3]]]))))+
                     np.log(1+ np.exp(-(np.array(x[start1:start2]).reshape(real_theta2.shape)-np.array([[x[start3:]]]))))-
                     (1-y)*np.log((1+ np.exp(-(np.array(x[:start1]).reshape(real_theta1.shape)-np.array([[x[start2:start3]]]))))*(1+ np.exp(-(np.array(x[start1:start2]).reshape(real_theta1.shape)-np.array([[x[start3:]]]))))-1)).sum()/n_items+w*((np.array([x])**2).sum())
#     fun = lambda x: (np.log(1+ np.exp(-(np.array([[ii] for ii in x[:start1]])-np.array([[x[start2:start3]]]))))+
#                      np.log(1+ np.exp(-(np.array([[ii] for ii in x[start1:start2]])-np.array([[x[start3:]]]))))-
#                      (1-y)*np.log((1+ np.exp(-(np.array([[ii] for ii in x[:start1]])-np.array([[x[start2:start3]]]))))*(1+ np.exp(-(np.array([[ii] for ii in x[start1:start2]])-np.array([[x[start3:]]]))))-1)).sum()/n_items+w*((np.array([[ii] for ii in x])**2).sum())
#     res = minimize(fun, (t0,t1,b0,b1), bounds=bnds,method='L-BFGS-B')
    res = minimize(fun, (a), bounds=bnds,method='L-BFGS-B')
    # print res.x
    t0_min = res.x[:start1]
    t1_min = res.x[start1:start2]
    b0_min = res.x[start2:start3]
    b1_min = res.x[start3:]
    print w,(absm(t0_min,t0_real)+absm(t1_min,t1_real)+absm(b0_min,b0_real)+absm(b1_min,b1_real))/4,absm(t0_min,t0_real),absm(t1_min,t1_real),absm(b0_min,b0_real),absm(b1_min,b1_real)

0.0 1.00891638722 1.0290779431 0.868336725078 1.01450014181 1.12375073887
0.001 0.572697846795 0.543431744709 0.623396436233 0.520258569799 0.603704636438
0.002 0.573661475191 0.546184671925 0.621536612091 0.526149134716 0.60077548203
0.003 0.57608313185 0.549365273785 0.620975658286 0.532319777638 0.60167181769
0.004 0.579515185994 0.552318023093 0.62259844989 0.539301984715 0.603842286278
0.005 0.583155593009 0.554962208229 0.625217103623 0.54623219764 0.606210862543
0.006 0.586665690637 0.557279695656 0.627810506473 0.553068741673 0.608503818747
0.007 0.590337767856 0.55975266762 0.630489174791 0.560019266064 0.611089962949
0.008 0.59408961223 0.562007861846 0.633297767498 0.566842201304 0.614210618273
0.009 0.597697858945 0.564317132835 0.635988425335 0.573339394106 0.617146483506
0.01 0.601375994291 0.567110061041 0.638516008186 0.579492938788 0.620384969148
0.011 0.605004880043 0.569921090625 0.640889271884 0.585430354143 0.623778803522
0.012 0.608544464449 0.572628495581 0.64343

In [94]:
n_items=400
n_stu=100
real_b1 = np.random.normal(0,1,(1,n_items))
real_theta1 = np.random.normal(0,1,(n_stu,1))
real_b2 = np.random.normal(0,1,(1,n_items))
real_theta2 = np.random.normal(0,1,(n_stu,1))
uv=np.random.random((n_stu,n_items))<1.0/ (1.0 + np.exp(-(real_theta1-real_b1)))/ (1.0 + np.exp(-(real_theta2-real_b2)))
def absm(a,b):
    return np.abs(a-b).mean()
b0_real = np.array([ee for ee in real_b1.flat])
b1_real = np.array([ee for ee in real_b2.flat])
t0_real = np.array([ee for ee in real_theta1.flat])
t1_real = np.array([ee for ee in real_theta2.flat])
a = np.array([1]*2*(n_stu+n_items))
start1 = n_stu
start2 = n_stu*2
start3 = n_stu*2+n_items
bnds = ((-3, 3), (-3, 3))*(n_stu+n_items)
y = uv
for w in [ii/1000. for ii in range(20)]:
    fun = lambda x: (np.log(1+ np.exp(-(np.array(x[:start1]).reshape(real_theta1.shape)-np.array([[x[start2:start3]]]))))+
                     np.log(1+ np.exp(-(np.array(x[start1:start2]).reshape(real_theta2.shape)-np.array([[x[start3:]]]))))-
                     (1-y)*np.log((1+ np.exp(-(np.array(x[:start1]).reshape(real_theta1.shape)-np.array([[x[start2:start3]]]))))*(1+ np.exp(-(np.array(x[start1:start2]).reshape(real_theta1.shape)-np.array([[x[start3:]]]))))-1)).sum()/n_items+w*((np.array([x])**2).sum())
#     fun = lambda x: (np.log(1+ np.exp(-(np.array([[ii] for ii in x[:start1]])-np.array([[x[start2:start3]]]))))+
#                      np.log(1+ np.exp(-(np.array([[ii] for ii in x[start1:start2]])-np.array([[x[start3:]]]))))-
#                      (1-y)*np.log((1+ np.exp(-(np.array([[ii] for ii in x[:start1]])-np.array([[x[start2:start3]]]))))*(1+ np.exp(-(np.array([[ii] for ii in x[start1:start2]])-np.array([[x[start3:]]]))))-1)).sum()/n_items+w*((np.array([[ii] for ii in x])**2).sum())
#     res = minimize(fun, (t0,t1,b0,b1), bounds=bnds,method='L-BFGS-B')
    res = minimize(fun, (a), bounds=bnds,method='L-BFGS-B')
    # print res.x
    t0_min = res.x[:start1]
    t1_min = res.x[start1:start2]
    b0_min = res.x[start2:start3]
    b1_min = res.x[start3:]
    print w,(absm(t0_min,t0_real)+absm(t1_min,t1_real)+absm(b0_min,b0_real)+absm(b1_min,b1_real))/4,absm(t0_min,t0_real),absm(t1_min,t1_real),absm(b0_min,b0_real),absm(b1_min,b1_real)

0.0 1.00000269576 1.00301587136 0.90806983842 1.05350444644 1.03542062683
0.001 0.588877896494 0.606423223265 0.577555871032 0.590834120172 0.580698371508
0.002 0.592060103738 0.612016456099 0.574991052351 0.600288041331 0.580944865172
0.003 0.594858165811 0.61695792294 0.571270770689 0.609543814003 0.581660155611
0.004 0.598517625946 0.620334140803 0.570437040122 0.618572995744 0.584726327116
0.005 0.602159784788 0.623647697657 0.569659109676 0.627162108108 0.588170223709
0.006 0.605768838549 0.626654990755 0.569110749107 0.635508839715 0.591800774621
0.007 0.609409042226 0.629337807515 0.569442931607 0.643196070268 0.595659359511
0.008 0.612911998972 0.63185210397 0.569862080509 0.650223960117 0.59970985129
0.009 0.616208243859 0.634141763476 0.570213545934 0.656692736657 0.603784929369
0.01 0.619422980024 0.636251821927 0.570558569053 0.662793733491 0.608087795625
0.011 0.622498957311 0.638200256546 0.57092426263 0.668571680058 0.612299630011
0.012 0.625400201559 0.640030688215 0.57

In [95]:
n_items=500
n_stu=100
real_b1 = np.random.normal(0,1,(1,n_items))
real_theta1 = np.random.normal(0,1,(n_stu,1))
real_b2 = np.random.normal(0,1,(1,n_items))
real_theta2 = np.random.normal(0,1,(n_stu,1))
uv=np.random.random((n_stu,n_items))<1.0/ (1.0 + np.exp(-(real_theta1-real_b1)))/ (1.0 + np.exp(-(real_theta2-real_b2)))
def absm(a,b):
    return np.abs(a-b).mean()
b0_real = np.array([ee for ee in real_b1.flat])
b1_real = np.array([ee for ee in real_b2.flat])
t0_real = np.array([ee for ee in real_theta1.flat])
t1_real = np.array([ee for ee in real_theta2.flat])
a = np.array([1]*2*(n_stu+n_items))
start1 = n_stu
start2 = n_stu*2
start3 = n_stu*2+n_items
bnds = ((-3, 3), (-3, 3))*(n_stu+n_items)
y = uv
for w in [ii/1000. for ii in range(20)]:
    fun = lambda x: (np.log(1+ np.exp(-(np.array(x[:start1]).reshape(real_theta1.shape)-np.array([[x[start2:start3]]]))))+
                     np.log(1+ np.exp(-(np.array(x[start1:start2]).reshape(real_theta2.shape)-np.array([[x[start3:]]]))))-
                     (1-y)*np.log((1+ np.exp(-(np.array(x[:start1]).reshape(real_theta1.shape)-np.array([[x[start2:start3]]]))))*(1+ np.exp(-(np.array(x[start1:start2]).reshape(real_theta1.shape)-np.array([[x[start3:]]]))))-1)).sum()/n_items+w*((np.array([x])**2).sum())
#     fun = lambda x: (np.log(1+ np.exp(-(np.array([[ii] for ii in x[:start1]])-np.array([[x[start2:start3]]]))))+
#                      np.log(1+ np.exp(-(np.array([[ii] for ii in x[start1:start2]])-np.array([[x[start3:]]]))))-
#                      (1-y)*np.log((1+ np.exp(-(np.array([[ii] for ii in x[:start1]])-np.array([[x[start2:start3]]]))))*(1+ np.exp(-(np.array([[ii] for ii in x[start1:start2]])-np.array([[x[start3:]]]))))-1)).sum()/n_items+w*((np.array([[ii] for ii in x])**2).sum())
#     res = minimize(fun, (t0,t1,b0,b1), bounds=bnds,method='L-BFGS-B')
    res = minimize(fun, (a), bounds=bnds,method='L-BFGS-B')
    # print res.x
    t0_min = res.x[:start1]
    t1_min = res.x[start1:start2]
    b0_min = res.x[start2:start3]
    b1_min = res.x[start3:]
    print w,(absm(t0_min,t0_real)+absm(t1_min,t1_real)+absm(b0_min,b0_real)+absm(b1_min,b1_real))/4,absm(t0_min,t0_real),absm(t1_min,t1_real),absm(b0_min,b0_real),absm(b1_min,b1_real)

0.0 1.04030594023 1.02239778877 0.947407968378 1.07384601479 1.11757198898
0.001 0.608251006327 0.600688294462 0.631614715937 0.589083848437 0.611617166471
0.002 0.608623563328 0.606574959742 0.628295892448 0.584863566582 0.614759834542
0.003 0.611844047998 0.612786921592 0.627053011964 0.586833373849 0.620702884588
0.004 0.615677042919 0.616677663299 0.626063748139 0.592255454495 0.627711305744
0.005 0.619837199564 0.619905006667 0.625183551413 0.59869307582 0.635567164356
0.006 0.623938377471 0.622729073781 0.62441379514 0.605472075278 0.643138565683
0.007 0.627655044002 0.625074688359 0.623834092428 0.611849134192 0.649862261029
0.008 0.631370032512 0.627584565208 0.62344437128 0.618465490775 0.655985702785
0.009 0.634848499437 0.62991478489 0.623083925898 0.624659715377 0.661735571584
0.01 0.638127034132 0.632062315928 0.622795479586 0.630543763153 0.66710657786
0.011 0.641275471298 0.634051057855 0.622768284312 0.636122964727 0.6721595783
0.012 0.644252249566 0.63593227451 0.62285

In [96]:
n_items=1000
n_stu=100
real_b1 = np.random.normal(0,1,(1,n_items))
real_theta1 = np.random.normal(0,1,(n_stu,1))
real_b2 = np.random.normal(0,1,(1,n_items))
real_theta2 = np.random.normal(0,1,(n_stu,1))
uv=np.random.random((n_stu,n_items))<1.0/ (1.0 + np.exp(-(real_theta1-real_b1)))/ (1.0 + np.exp(-(real_theta2-real_b2)))
def absm(a,b):
    return np.abs(a-b).mean()
b0_real = np.array([ee for ee in real_b1.flat])
b1_real = np.array([ee for ee in real_b2.flat])
t0_real = np.array([ee for ee in real_theta1.flat])
t1_real = np.array([ee for ee in real_theta2.flat])
a = np.array([1]*2*(n_stu+n_items))
start1 = n_stu
start2 = n_stu*2
start3 = n_stu*2+n_items
bnds = ((-3, 3), (-3, 3))*(n_stu+n_items)
y = uv
for w in [ii/1000. for ii in range(20)]:
    fun = lambda x: (np.log(1+ np.exp(-(np.array(x[:start1]).reshape(real_theta1.shape)-np.array([[x[start2:start3]]]))))+
                     np.log(1+ np.exp(-(np.array(x[start1:start2]).reshape(real_theta2.shape)-np.array([[x[start3:]]]))))-
                     (1-y)*np.log((1+ np.exp(-(np.array(x[:start1]).reshape(real_theta1.shape)-np.array([[x[start2:start3]]]))))*(1+ np.exp(-(np.array(x[start1:start2]).reshape(real_theta1.shape)-np.array([[x[start3:]]]))))-1)).sum()/n_items+w*((np.array([x])**2).sum())
#     fun = lambda x: (np.log(1+ np.exp(-(np.array([[ii] for ii in x[:start1]])-np.array([[x[start2:start3]]]))))+
#                      np.log(1+ np.exp(-(np.array([[ii] for ii in x[start1:start2]])-np.array([[x[start3:]]]))))-
#                      (1-y)*np.log((1+ np.exp(-(np.array([[ii] for ii in x[:start1]])-np.array([[x[start2:start3]]]))))*(1+ np.exp(-(np.array([[ii] for ii in x[start1:start2]])-np.array([[x[start3:]]]))))-1)).sum()/n_items+w*((np.array([[ii] for ii in x])**2).sum())
#     res = minimize(fun, (t0,t1,b0,b1), bounds=bnds,method='L-BFGS-B')
    res = minimize(fun, (a), bounds=bnds,method='L-BFGS-B')
    # print res.x
    t0_min = res.x[:start1]
    t1_min = res.x[start1:start2]
    b0_min = res.x[start2:start3]
    b1_min = res.x[start3:]
    print w,(absm(t0_min,t0_real)+absm(t1_min,t1_real)+absm(b0_min,b0_real)+absm(b1_min,b1_real))/4,absm(t0_min,t0_real),absm(t1_min,t1_real),absm(b0_min,b0_real),absm(b1_min,b1_real)

0.0 1.03445258018 0.999918027388 0.960274921887 1.06521113647 1.11240623499
0.001 0.916376148415 0.889837339172 0.861248847985 0.932250624106 0.982167782395
0.002 0.801398208342 0.776729713118 0.761847094316 0.810022355888 0.856993670046
0.003 0.720775935404 0.700009666092 0.706214528933 0.715713405256 0.761166141334
0.004 0.677786341508 0.658483787686 0.680506106463 0.663996705037 0.708158766846
0.005 0.656441670769 0.639094051284 0.668772662133 0.637670940318 0.680229029343
0.006 0.635199921902 0.613513867213 0.656464513483 0.613561546249 0.657259760664
0.007 0.632120911536 0.606708406842 0.653862554148 0.612709932928 0.655202752228
0.008 0.636397205248 0.603677125684 0.661137584428 0.62111560477 0.659658506111
0.009 0.644376607762 0.605425798125 0.669386347023 0.634577116683 0.668117169216
0.01 0.654520193134 0.611753169408 0.66966712274 0.652249784042 0.684410696348
0.011 0.659633983659 0.614474055459 0.673062433231 0.660065255828 0.690934190119
0.012 0.664489987969 0.617876174205 

In [97]:
n_items=80
n_stu=100
real_b1 = np.random.normal(0,1,(1,n_items))
real_theta1 = np.random.normal(0,1,(n_stu,1))
real_b2 = np.random.normal(0,1,(1,n_items))
real_theta2 = np.random.normal(0,1,(n_stu,1))
uv=np.random.random((n_stu,n_items))<1.0/ (1.0 + np.exp(-(real_theta1-real_b1)))/ (1.0 + np.exp(-(real_theta2-real_b2)))
def absm(a,b):
    return np.abs(a-b).mean()
b0_real = np.array([ee for ee in real_b1.flat])
b1_real = np.array([ee for ee in real_b2.flat])
t0_real = np.array([ee for ee in real_theta1.flat])
t1_real = np.array([ee for ee in real_theta2.flat])
a = np.array([1]*2*(n_stu+n_items))
start1 = n_stu
start2 = n_stu*2
start3 = n_stu*2+n_items
bnds = ((-3, 3), (-3, 3))*(n_stu+n_items)
y = uv
for w in [ii/1000. for ii in range(10)]:
    fun = lambda x: (np.log(1+ np.exp(-(np.array(x[:start1]).reshape(real_theta1.shape)-np.array([[x[start2:start3]]]))))+
                     np.log(1+ np.exp(-(np.array(x[start1:start2]).reshape(real_theta2.shape)-np.array([[x[start3:]]]))))-
                     (1-y)*np.log((1+ np.exp(-(np.array(x[:start1]).reshape(real_theta1.shape)-np.array([[x[start2:start3]]]))))*(1+ np.exp(-(np.array(x[start1:start2]).reshape(real_theta1.shape)-np.array([[x[start3:]]]))))-1)).sum()/n_items+w*((np.array([x])**2).sum())
#     fun = lambda x: (np.log(1+ np.exp(-(np.array([[ii] for ii in x[:start1]])-np.array([[x[start2:start3]]]))))+
#                      np.log(1+ np.exp(-(np.array([[ii] for ii in x[start1:start2]])-np.array([[x[start3:]]]))))-
#                      (1-y)*np.log((1+ np.exp(-(np.array([[ii] for ii in x[:start1]])-np.array([[x[start2:start3]]]))))*(1+ np.exp(-(np.array([[ii] for ii in x[start1:start2]])-np.array([[x[start3:]]]))))-1)).sum()/n_items+w*((np.array([[ii] for ii in x])**2).sum())
#     res = minimize(fun, (t0,t1,b0,b1), bounds=bnds,method='L-BFGS-B')
    res = minimize(fun, (a), bounds=bnds,method='L-BFGS-B')
    # print res.x
    t0_min = res.x[:start1]
    t1_min = res.x[start1:start2]
    b0_min = res.x[start2:start3]
    b1_min = res.x[start3:]
    print w,(absm(t0_min,t0_real)+absm(t1_min,t1_real)+absm(b0_min,b0_real)+absm(b1_min,b1_real))/4,absm(t0_min,t0_real),absm(t1_min,t1_real),absm(b0_min,b0_real),absm(b1_min,b1_real)

0.0 1.06641710868 1.01502203441 1.03507444296 1.12935921007 1.08621274727
0.001 0.595438126421 0.601455789277 0.657751698736 0.55499848828 0.567546529394
0.002 0.585240066087 0.591137278688 0.659450639661 0.547958529381 0.542413816616
0.003 0.585084910427 0.592870606111 0.65762947465 0.547141754328 0.542697806617
0.004 0.585152602768 0.594825538785 0.656370146657 0.546408275024 0.543006450605
0.005 0.585323971081 0.596765182355 0.655415869473 0.545772798939 0.543342033558
0.006 0.585779945525 0.598540352438 0.654675424545 0.545805730381 0.544098274736
0.007 0.586347247937 0.600129068361 0.654095336094 0.546122662717 0.545041924575
0.008 0.586967319464 0.601566285879 0.653828677994 0.546433295809 0.546041018175
0.009 0.587669241716 0.602893685013 0.653647131741 0.547038963106 0.547097187003


In [98]:
n_items=60
n_stu=100
real_b1 = np.random.normal(0,1,(1,n_items))
real_theta1 = np.random.normal(0,1,(n_stu,1))
real_b2 = np.random.normal(0,1,(1,n_items))
real_theta2 = np.random.normal(0,1,(n_stu,1))
uv=np.random.random((n_stu,n_items))<1.0/ (1.0 + np.exp(-(real_theta1-real_b1)))/ (1.0 + np.exp(-(real_theta2-real_b2)))
def absm(a,b):
    return np.abs(a-b).mean()
b0_real = np.array([ee for ee in real_b1.flat])
b1_real = np.array([ee for ee in real_b2.flat])
t0_real = np.array([ee for ee in real_theta1.flat])
t1_real = np.array([ee for ee in real_theta2.flat])
a = np.array([1]*2*(n_stu+n_items))
start1 = n_stu
start2 = n_stu*2
start3 = n_stu*2+n_items
bnds = ((-3, 3), (-3, 3))*(n_stu+n_items)
y = uv
for w in [ii/1000. for ii in range(10)]:
    fun = lambda x: (np.log(1+ np.exp(-(np.array(x[:start1]).reshape(real_theta1.shape)-np.array([[x[start2:start3]]]))))+
                     np.log(1+ np.exp(-(np.array(x[start1:start2]).reshape(real_theta2.shape)-np.array([[x[start3:]]]))))-
                     (1-y)*np.log((1+ np.exp(-(np.array(x[:start1]).reshape(real_theta1.shape)-np.array([[x[start2:start3]]]))))*(1+ np.exp(-(np.array(x[start1:start2]).reshape(real_theta1.shape)-np.array([[x[start3:]]]))))-1)).sum()/n_items+w*((np.array([x])**2).sum())
#     fun = lambda x: (np.log(1+ np.exp(-(np.array([[ii] for ii in x[:start1]])-np.array([[x[start2:start3]]]))))+
#                      np.log(1+ np.exp(-(np.array([[ii] for ii in x[start1:start2]])-np.array([[x[start3:]]]))))-
#                      (1-y)*np.log((1+ np.exp(-(np.array([[ii] for ii in x[:start1]])-np.array([[x[start2:start3]]]))))*(1+ np.exp(-(np.array([[ii] for ii in x[start1:start2]])-np.array([[x[start3:]]]))))-1)).sum()/n_items+w*((np.array([[ii] for ii in x])**2).sum())
#     res = minimize(fun, (t0,t1,b0,b1), bounds=bnds,method='L-BFGS-B')
    res = minimize(fun, (a), bounds=bnds,method='L-BFGS-B')
    # print res.x
    t0_min = res.x[:start1]
    t1_min = res.x[start1:start2]
    b0_min = res.x[start2:start3]
    b1_min = res.x[start3:]
    print w,(absm(t0_min,t0_real)+absm(t1_min,t1_real)+absm(b0_min,b0_real)+absm(b1_min,b1_real))/4,absm(t0_min,t0_real),absm(t1_min,t1_real),absm(b0_min,b0_real),absm(b1_min,b1_real)

0.0 1.12297728564 0.927368812703 1.13706022703 1.18713481893 1.24034528391
0.001 0.582813150634 0.530081820399 0.592041459225 0.630633101629 0.578496221281
0.002 0.948928048545 0.956005905441 0.975563825119 0.979913507913 0.884228955707
0.003 0.582011954758 0.530752504858 0.589739106077 0.629065601401 0.578490606695
0.004 0.58206067544 0.531625755493 0.588808542494 0.628690924959 0.579117478813
0.005 0.582532331488 0.532619200623 0.588532995728 0.628391086343 0.580586043258
0.006 0.582998957737 0.533538514839 0.588347412186 0.628132311774 0.581977592148
0.007 0.583471068386 0.534413751448 0.588192769859 0.627927949352 0.583349802886
0.008 0.584079166429 0.535272500368 0.588177329115 0.627762932167 0.585103904064
0.009 0.584736398011 0.536139304892 0.588247507008 0.627608723528 0.586950056615


In [99]:
n_items=50
n_stu=100
real_b1 = np.random.normal(0,1,(1,n_items))
real_theta1 = np.random.normal(0,1,(n_stu,1))
real_b2 = np.random.normal(0,1,(1,n_items))
real_theta2 = np.random.normal(0,1,(n_stu,1))
uv=np.random.random((n_stu,n_items))<1.0/ (1.0 + np.exp(-(real_theta1-real_b1)))/ (1.0 + np.exp(-(real_theta2-real_b2)))
def absm(a,b):
    return np.abs(a-b).mean()
b0_real = np.array([ee for ee in real_b1.flat])
b1_real = np.array([ee for ee in real_b2.flat])
t0_real = np.array([ee for ee in real_theta1.flat])
t1_real = np.array([ee for ee in real_theta2.flat])
a = np.array([1]*2*(n_stu+n_items))
start1 = n_stu
start2 = n_stu*2
start3 = n_stu*2+n_items
bnds = ((-3, 3), (-3, 3))*(n_stu+n_items)
y = uv
for w in [ii/1000. for ii in range(20)]:
    fun = lambda x: (np.log(1+ np.exp(-(np.array(x[:start1]).reshape(real_theta1.shape)-np.array([[x[start2:start3]]]))))+
                     np.log(1+ np.exp(-(np.array(x[start1:start2]).reshape(real_theta2.shape)-np.array([[x[start3:]]]))))-
                     (1-y)*np.log((1+ np.exp(-(np.array(x[:start1]).reshape(real_theta1.shape)-np.array([[x[start2:start3]]]))))*(1+ np.exp(-(np.array(x[start1:start2]).reshape(real_theta1.shape)-np.array([[x[start3:]]]))))-1)).sum()/n_items+w*((np.array([x])**2).sum())
#     fun = lambda x: (np.log(1+ np.exp(-(np.array([[ii] for ii in x[:start1]])-np.array([[x[start2:start3]]]))))+
#                      np.log(1+ np.exp(-(np.array([[ii] for ii in x[start1:start2]])-np.array([[x[start3:]]]))))-
#                      (1-y)*np.log((1+ np.exp(-(np.array([[ii] for ii in x[:start1]])-np.array([[x[start2:start3]]]))))*(1+ np.exp(-(np.array([[ii] for ii in x[start1:start2]])-np.array([[x[start3:]]]))))-1)).sum()/n_items+w*((np.array([[ii] for ii in x])**2).sum())
#     res = minimize(fun, (t0,t1,b0,b1), bounds=bnds,method='L-BFGS-B')
    res = minimize(fun, (a), bounds=bnds,method='L-BFGS-B')
    # print res.x
    t0_min = res.x[:start1]
    t1_min = res.x[start1:start2]
    b0_min = res.x[start2:start3]
    b1_min = res.x[start3:]
    print w,(absm(t0_min,t0_real)+absm(t1_min,t1_real)+absm(b0_min,b0_real)+absm(b1_min,b1_real))/4,absm(t0_min,t0_real),absm(t1_min,t1_real),absm(b0_min,b0_real),absm(b1_min,b1_real)

0.0 0.715024957342 0.680652705722 0.659466190204 0.684231806529 0.835749126912
0.001 1.11585600803 0.986957437556 1.0361824931 1.19879985836 1.24148424311
0.002 0.830009838686 0.766647984583 0.809579647787 0.871769849337 0.872041873037
0.003 1.05034760303 0.916779222472 0.986951230397 1.13048101692 1.16717894232
0.004 0.6129522401 0.611481906805 0.576217119801 0.630725308114 0.633384625679
0.005 0.613004583849 0.6099555345 0.576343712472 0.632847095369 0.632871993055
0.006 0.613144976471 0.608491632512 0.576556458429 0.634878293492 0.632653521451
0.007 0.613395891765 0.607237887137 0.576930074958 0.636733623247 0.632681981718
0.008 0.613796191884 0.606550963794 0.57743110623 0.638432645943 0.632770051571
0.009 0.61429050595 0.606228412052 0.5780281586 0.640025720552 0.632879732597
0.01 0.614763442869 0.605925677917 0.578589358769 0.64150271031 0.63303602448
0.011 0.61524802252 0.605766480918 0.579118863309 0.642889480506 0.633217265345
0.012 0.615847128786 0.606033875547 0.579749229237

In [100]:
n_items=40
n_stu=100
real_b1 = np.random.normal(0,1,(1,n_items))
real_theta1 = np.random.normal(0,1,(n_stu,1))
real_b2 = np.random.normal(0,1,(1,n_items))
real_theta2 = np.random.normal(0,1,(n_stu,1))
uv=np.random.random((n_stu,n_items))<1.0/ (1.0 + np.exp(-(real_theta1-real_b1)))/ (1.0 + np.exp(-(real_theta2-real_b2)))
def absm(a,b):
    return np.abs(a-b).mean()
b0_real = np.array([ee for ee in real_b1.flat])
b1_real = np.array([ee for ee in real_b2.flat])
t0_real = np.array([ee for ee in real_theta1.flat])
t1_real = np.array([ee for ee in real_theta2.flat])
a = np.array([1]*2*(n_stu+n_items))
start1 = n_stu
start2 = n_stu*2
start3 = n_stu*2+n_items
bnds = ((-3, 3), (-3, 3))*(n_stu+n_items)
y = uv
for w in [ii/1000. for ii in range(20)]:
    fun = lambda x: (np.log(1+ np.exp(-(np.array(x[:start1]).reshape(real_theta1.shape)-np.array([[x[start2:start3]]]))))+
                     np.log(1+ np.exp(-(np.array(x[start1:start2]).reshape(real_theta2.shape)-np.array([[x[start3:]]]))))-
                     (1-y)*np.log((1+ np.exp(-(np.array(x[:start1]).reshape(real_theta1.shape)-np.array([[x[start2:start3]]]))))*(1+ np.exp(-(np.array(x[start1:start2]).reshape(real_theta1.shape)-np.array([[x[start3:]]]))))-1)).sum()/n_items+w*((np.array([x])**2).sum())
#     fun = lambda x: (np.log(1+ np.exp(-(np.array([[ii] for ii in x[:start1]])-np.array([[x[start2:start3]]]))))+
#                      np.log(1+ np.exp(-(np.array([[ii] for ii in x[start1:start2]])-np.array([[x[start3:]]]))))-
#                      (1-y)*np.log((1+ np.exp(-(np.array([[ii] for ii in x[:start1]])-np.array([[x[start2:start3]]]))))*(1+ np.exp(-(np.array([[ii] for ii in x[start1:start2]])-np.array([[x[start3:]]]))))-1)).sum()/n_items+w*((np.array([[ii] for ii in x])**2).sum())
#     res = minimize(fun, (t0,t1,b0,b1), bounds=bnds,method='L-BFGS-B')
    res = minimize(fun, (a), bounds=bnds,method='L-BFGS-B')
    # print res.x
    t0_min = res.x[:start1]
    t1_min = res.x[start1:start2]
    b0_min = res.x[start2:start3]
    b1_min = res.x[start3:]
    print w,(absm(t0_min,t0_real)+absm(t1_min,t1_real)+absm(b0_min,b0_real)+absm(b1_min,b1_real))/4,absm(t0_min,t0_real),absm(t1_min,t1_real),absm(b0_min,b0_real),absm(b1_min,b1_real)

0.0 0.999210049245 1.03219295322 0.923991319562 1.10923908436 0.931416839835
0.001 1.23947351757 1.14716258948 1.24881898475 1.22086056213 1.34105193391
0.002 0.593601065799 0.662748532736 0.59016556632 0.594173598784 0.527316565356
0.003 0.589397403361 0.653831880269 0.584247372585 0.591984644757 0.527525715831
0.004 1.10512239288 1.00575053375 0.990574948058 1.22639215469 1.19777193504
0.005 0.58403650003 0.643138155468 0.576519639329 0.588469975331 0.528018229994
0.006 0.582134743022 0.639122041447 0.573912898808 0.58698183597 0.528522195861
0.007 0.580961514799 0.637051769007 0.57210235837 0.585605921949 0.52908600987
0.008 0.580546788719 0.636098204264 0.572143822658 0.584326175854 0.529618952102
0.009 0.580378886064 0.63532836093 0.572460670629 0.583589442374 0.530137070322
0.01 0.580400776401 0.63487276916 0.572767401303 0.583325229831 0.530637705311
0.011 0.580494971959 0.63446425034 0.573067855477 0.583332166729 0.53111561529
0.012 0.580602369811 0.634114251863 0.57336525621 0

In [101]:
n_items=30
n_stu=100
real_b1 = np.random.normal(0,1,(1,n_items))
real_theta1 = np.random.normal(0,1,(n_stu,1))
real_b2 = np.random.normal(0,1,(1,n_items))
real_theta2 = np.random.normal(0,1,(n_stu,1))
uv=np.random.random((n_stu,n_items))<1.0/ (1.0 + np.exp(-(real_theta1-real_b1)))/ (1.0 + np.exp(-(real_theta2-real_b2)))
def absm(a,b):
    return np.abs(a-b).mean()
b0_real = np.array([ee for ee in real_b1.flat])
b1_real = np.array([ee for ee in real_b2.flat])
t0_real = np.array([ee for ee in real_theta1.flat])
t1_real = np.array([ee for ee in real_theta2.flat])
a = np.array([1]*2*(n_stu+n_items))
start1 = n_stu
start2 = n_stu*2
start3 = n_stu*2+n_items
bnds = ((-3, 3), (-3, 3))*(n_stu+n_items)
y = uv
for w in [ii/1000. for ii in range(20)]:
    fun = lambda x: (np.log(1+ np.exp(-(np.array(x[:start1]).reshape(real_theta1.shape)-np.array([[x[start2:start3]]]))))+
                     np.log(1+ np.exp(-(np.array(x[start1:start2]).reshape(real_theta2.shape)-np.array([[x[start3:]]]))))-
                     (1-y)*np.log((1+ np.exp(-(np.array(x[:start1]).reshape(real_theta1.shape)-np.array([[x[start2:start3]]]))))*(1+ np.exp(-(np.array(x[start1:start2]).reshape(real_theta1.shape)-np.array([[x[start3:]]]))))-1)).sum()/n_items+w*((np.array([x])**2).sum())
#     fun = lambda x: (np.log(1+ np.exp(-(np.array([[ii] for ii in x[:start1]])-np.array([[x[start2:start3]]]))))+
#                      np.log(1+ np.exp(-(np.array([[ii] for ii in x[start1:start2]])-np.array([[x[start3:]]]))))-
#                      (1-y)*np.log((1+ np.exp(-(np.array([[ii] for ii in x[:start1]])-np.array([[x[start2:start3]]]))))*(1+ np.exp(-(np.array([[ii] for ii in x[start1:start2]])-np.array([[x[start3:]]]))))-1)).sum()/n_items+w*((np.array([[ii] for ii in x])**2).sum())
#     res = minimize(fun, (t0,t1,b0,b1), bounds=bnds,method='L-BFGS-B')
    res = minimize(fun, (a), bounds=bnds,method='L-BFGS-B')
    # print res.x
    t0_min = res.x[:start1]
    t1_min = res.x[start1:start2]
    b0_min = res.x[start2:start3]
    b1_min = res.x[start3:]
    print w,(absm(t0_min,t0_real)+absm(t1_min,t1_real)+absm(b0_min,b0_real)+absm(b1_min,b1_real))/4,absm(t0_min,t0_real),absm(t1_min,t1_real),absm(b0_min,b0_real),absm(b1_min,b1_real)

0.0 1.07171250023 0.97346790416 1.04662819126 1.06119053537 1.20556337012
0.001 1.11298682238 1.04206009934 1.04399434217 1.09829184192 1.26760100609
0.002 1.4147262729 1.23591101184 1.27678434378 1.41604576921 1.73016396678
0.003 1.34853922132 1.20319140156 1.2062558689 1.41808152408 1.56662809071
0.004 1.35135803769 1.13755141778 1.18392651066 1.42401718496 1.65993703737
0.005 0.685412817347 0.682130749657 0.67854981523 0.63179912185 0.749171582652
0.006 0.682456208726 0.676282072741 0.675798645166 0.634057432644 0.743686684352
0.007 0.680375949073 0.672437504061 0.674196372483 0.636215250005 0.738654669741
0.008 0.678926488384 0.67014056436 0.673153495654 0.638172617532 0.734239275991
0.009 0.677668655631 0.66819848023 0.672222785446 0.640131328549 0.730122028299
0.01 0.676787847044 0.666457708098 0.671346783439 0.641976048184 0.727370848454
0.011 0.675960549915 0.664783894427 0.670520320605 0.643727906483 0.724810078146
0.012 0.675293386077 0.663641681491 0.669742423451 0.645407255

In [102]:
n_items=20
n_stu=100
real_b1 = np.random.normal(0,1,(1,n_items))
real_theta1 = np.random.normal(0,1,(n_stu,1))
real_b2 = np.random.normal(0,1,(1,n_items))
real_theta2 = np.random.normal(0,1,(n_stu,1))
uv=np.random.random((n_stu,n_items))<1.0/ (1.0 + np.exp(-(real_theta1-real_b1)))/ (1.0 + np.exp(-(real_theta2-real_b2)))
def absm(a,b):
    return np.abs(a-b).mean()
b0_real = np.array([ee for ee in real_b1.flat])
b1_real = np.array([ee for ee in real_b2.flat])
t0_real = np.array([ee for ee in real_theta1.flat])
t1_real = np.array([ee for ee in real_theta2.flat])
a = np.array([1]*2*(n_stu+n_items))
start1 = n_stu
start2 = n_stu*2
start3 = n_stu*2+n_items
bnds = ((-3, 3), (-3, 3))*(n_stu+n_items)
y = uv
for w in [ii/1000. for ii in range(20)]:
    fun = lambda x: (np.log(1+ np.exp(-(np.array(x[:start1]).reshape(real_theta1.shape)-np.array([[x[start2:start3]]]))))+
                     np.log(1+ np.exp(-(np.array(x[start1:start2]).reshape(real_theta2.shape)-np.array([[x[start3:]]]))))-
                     (1-y)*np.log((1+ np.exp(-(np.array(x[:start1]).reshape(real_theta1.shape)-np.array([[x[start2:start3]]]))))*(1+ np.exp(-(np.array(x[start1:start2]).reshape(real_theta1.shape)-np.array([[x[start3:]]]))))-1)).sum()/n_items+w*((np.array([x])**2).sum())
#     fun = lambda x: (np.log(1+ np.exp(-(np.array([[ii] for ii in x[:start1]])-np.array([[x[start2:start3]]]))))+
#                      np.log(1+ np.exp(-(np.array([[ii] for ii in x[start1:start2]])-np.array([[x[start3:]]]))))-
#                      (1-y)*np.log((1+ np.exp(-(np.array([[ii] for ii in x[:start1]])-np.array([[x[start2:start3]]]))))*(1+ np.exp(-(np.array([[ii] for ii in x[start1:start2]])-np.array([[x[start3:]]]))))-1)).sum()/n_items+w*((np.array([[ii] for ii in x])**2).sum())
#     res = minimize(fun, (t0,t1,b0,b1), bounds=bnds,method='L-BFGS-B')
    res = minimize(fun, (a), bounds=bnds,method='L-BFGS-B')
    # print res.x
    t0_min = res.x[:start1]
    t1_min = res.x[start1:start2]
    b0_min = res.x[start2:start3]
    b1_min = res.x[start3:]
    print w,(absm(t0_min,t0_real)+absm(t1_min,t1_real)+absm(b0_min,b0_real)+absm(b1_min,b1_real))/4,absm(t0_min,t0_real),absm(t1_min,t1_real),absm(b0_min,b0_real),absm(b1_min,b1_real)

0.0 1.33654643606 1.35934207523 1.32539254779 1.23707070764 1.42438041359
0.001 1.57363504762 1.31410594364 1.48981470364 1.62998226757 1.86063727561
0.002 1.12893486271 1.12757881325 1.0248702453 1.29301899967 1.07027139263
0.003 1.23661024073 0.940593154051 1.07610615404 1.44352474388 1.48621691095
0.004 1.24622926529 1.21204215452 1.06429762462 1.39563887387 1.31293840814
0.005 1.10519245762 0.96782179797 1.00210991326 1.24096954881 1.20986857043
0.006 0.619107116985 0.671141362465 0.638923375396 0.587849968354 0.578513761725
0.007 1.0719605187 0.909310847943 0.891277215387 1.20947696677 1.27777704471
0.008 0.6192199084 0.670227769517 0.638207682536 0.587301965178 0.58114221637
0.009 0.61950861403 0.67023517305 0.638452724139 0.587073434997 0.582273123933
0.01 0.619898990719 0.670503665148 0.638732177837 0.586898131185 0.583461988706
0.011 0.620328322934 0.671065337386 0.638975966379 0.586691551079 0.584580436892
0.012 0.620852053101 0.671930826601 0.639342755497 0.586495560637 0.58

In [103]:
n_items=10
n_stu=100
real_b1 = np.random.normal(0,1,(1,n_items))
real_theta1 = np.random.normal(0,1,(n_stu,1))
real_b2 = np.random.normal(0,1,(1,n_items))
real_theta2 = np.random.normal(0,1,(n_stu,1))
uv=np.random.random((n_stu,n_items))<1.0/ (1.0 + np.exp(-(real_theta1-real_b1)))/ (1.0 + np.exp(-(real_theta2-real_b2)))
def absm(a,b):
    return np.abs(a-b).mean()
b0_real = np.array([ee for ee in real_b1.flat])
b1_real = np.array([ee for ee in real_b2.flat])
t0_real = np.array([ee for ee in real_theta1.flat])
t1_real = np.array([ee for ee in real_theta2.flat])
a = np.array([1]*2*(n_stu+n_items))
start1 = n_stu
start2 = n_stu*2
start3 = n_stu*2+n_items
bnds = ((-3, 3), (-3, 3))*(n_stu+n_items)
y = uv
for w in [ii/1000. for ii in range(20)]:
    fun = lambda x: (np.log(1+ np.exp(-(np.array(x[:start1]).reshape(real_theta1.shape)-np.array([[x[start2:start3]]]))))+
                     np.log(1+ np.exp(-(np.array(x[start1:start2]).reshape(real_theta2.shape)-np.array([[x[start3:]]]))))-
                     (1-y)*np.log((1+ np.exp(-(np.array(x[:start1]).reshape(real_theta1.shape)-np.array([[x[start2:start3]]]))))*(1+ np.exp(-(np.array(x[start1:start2]).reshape(real_theta1.shape)-np.array([[x[start3:]]]))))-1)).sum()/n_items+w*((np.array([x])**2).sum())
#     fun = lambda x: (np.log(1+ np.exp(-(np.array([[ii] for ii in x[:start1]])-np.array([[x[start2:start3]]]))))+
#                      np.log(1+ np.exp(-(np.array([[ii] for ii in x[start1:start2]])-np.array([[x[start3:]]]))))-
#                      (1-y)*np.log((1+ np.exp(-(np.array([[ii] for ii in x[:start1]])-np.array([[x[start2:start3]]]))))*(1+ np.exp(-(np.array([[ii] for ii in x[start1:start2]])-np.array([[x[start3:]]]))))-1)).sum()/n_items+w*((np.array([[ii] for ii in x])**2).sum())
#     res = minimize(fun, (t0,t1,b0,b1), bounds=bnds,method='L-BFGS-B')
    res = minimize(fun, (a), bounds=bnds,method='L-BFGS-B')
    # print res.x
    t0_min = res.x[:start1]
    t1_min = res.x[start1:start2]
    b0_min = res.x[start2:start3]
    b1_min = res.x[start3:]
    print w,(absm(t0_min,t0_real)+absm(t1_min,t1_real)+absm(b0_min,b0_real)+absm(b1_min,b1_real))/4,absm(t0_min,t0_real),absm(t1_min,t1_real),absm(b0_min,b0_real),absm(b1_min,b1_real)

0.0 1.11457624571 1.20100299548 1.31071052512 0.71175477153 1.23483669072
0.001 1.8279711862 1.58424008323 1.70312510248 1.89858932628 2.1259302328
0.002 1.35379137316 1.37762562806 1.40501773038 1.32392922878 1.30859290541
0.003 1.65685277436 1.49142866544 1.40317489175 1.93071487101 1.80209266924
0.004 1.70878582585 1.30155585183 1.36175988715 2.15258102448 2.01924653992
0.005 1.52034048473 1.18185513988 1.38452157102 1.50164794579 2.01333728222
0.006 1.16949742331 1.09224492585 1.21091251265 1.22541764863 1.1494146061
0.007 1.54887110545 1.22585184518 1.13918497911 1.97627832004 1.85416927747
0.008 1.32451511327 1.1030210529 1.15721471526 1.35993756546 1.67788711946
0.009 0.699119303727 0.702169304415 0.775323070266 0.547938106564 0.771046733662
0.01 1.40909733584 0.968384910405 1.03112793056 1.68661308681 1.9502634156
0.011 1.23191089782 0.902333347535 1.07964075845 1.3566002306 1.58906925468
0.012 0.686409479373 0.684294658108 0.749939577136 0.55518140667 0.756222275579
0.013 0.68

In [104]:
n_items=5
n_stu=100
real_b1 = np.random.normal(0,1,(1,n_items))
real_theta1 = np.random.normal(0,1,(n_stu,1))
real_b2 = np.random.normal(0,1,(1,n_items))
real_theta2 = np.random.normal(0,1,(n_stu,1))
uv=np.random.random((n_stu,n_items))<1.0/ (1.0 + np.exp(-(real_theta1-real_b1)))/ (1.0 + np.exp(-(real_theta2-real_b2)))
def absm(a,b):
    return np.abs(a-b).mean()
b0_real = np.array([ee for ee in real_b1.flat])
b1_real = np.array([ee for ee in real_b2.flat])
t0_real = np.array([ee for ee in real_theta1.flat])
t1_real = np.array([ee for ee in real_theta2.flat])
a = np.array([1]*2*(n_stu+n_items))
start1 = n_stu
start2 = n_stu*2
start3 = n_stu*2+n_items
bnds = ((-3, 3), (-3, 3))*(n_stu+n_items)
y = uv
for w in [ii/1000. for ii in range(20)]:
    fun = lambda x: (np.log(1+ np.exp(-(np.array(x[:start1]).reshape(real_theta1.shape)-np.array([[x[start2:start3]]]))))+
                     np.log(1+ np.exp(-(np.array(x[start1:start2]).reshape(real_theta2.shape)-np.array([[x[start3:]]]))))-
                     (1-y)*np.log((1+ np.exp(-(np.array(x[:start1]).reshape(real_theta1.shape)-np.array([[x[start2:start3]]]))))*(1+ np.exp(-(np.array(x[start1:start2]).reshape(real_theta1.shape)-np.array([[x[start3:]]]))))-1)).sum()/n_items+w*((np.array([x])**2).sum())
#     res = minimize(fun, (t0,t1,b0,b1), bounds=bnds,method='L-BFGS-B')
    res = minimize(fun, (a), bounds=bnds,method='L-BFGS-B')
    # print res.x
    t0_min = res.x[:start1]
    t1_min = res.x[start1:start2]
    b0_min = res.x[start2:start3]
    b1_min = res.x[start3:]
    print w,(absm(t0_min,t0_real)+absm(t1_min,t1_real)+absm(b0_min,b0_real)+absm(b1_min,b1_real))/4,absm(t0_min,t0_real),absm(t1_min,t1_real),absm(b0_min,b0_real),absm(b1_min,b1_real)

0.0 1.20556067997 1.45077917157 1.56477235391 0.797118870808 1.00957232359
0.001 1.59989443679 1.91888728382 1.82789622331 1.14260293209 1.51019130794
0.002 1.52685345568 1.82498961416 1.92413616937 1.13403071213 1.22425732706
0.003 1.55162136899 1.69531265664 1.80255496714 1.15354491709 1.55507293511
0.004 1.90500250208 1.62172221454 1.68863905896 2.44466770771 1.86498102711
0.005 1.46796373676 1.53742316621 1.63176759681 1.28389877565 1.41876540837
0.006 1.82105115931 1.48688018747 1.54105410334 2.44097111491 1.81529923152
0.007 1.32634746729 1.48092228187 1.49552870171 0.875077473972 1.45386141159
0.008 1.23093857274 1.48236008636 1.33005672726 0.787770660137 1.3235668172
0.009 1.15315919338 1.31332731164 1.2904574784 0.731294484237 1.27755749922
0.01 1.29294843689 1.31983876546 1.33399295649 0.956928146052 1.56103387957
0.011 1.14035053611 1.27690286371 1.20799915814 0.801962121614 1.27453800097
0.012 1.67098992239 1.18903077885 1.33133216867 2.42547453721 1.73812220482
0.013 1.695